# Pre-processing pipeline for spikeglx sessions, zebra finch
- For every run in the session:
 - Load the recordings
 - Extract wav chan with micrhopohone and make a wav chan with the nidq syn signal
 - Get the sync events for the nidq sync channel
 
 - Do bout detection
 
In another notebook, bout detection is curated
- Left to decide where to:
    - Sort spikes
    - Sync the spikes/lfp/nidq
    - make and plot 'bout rasters'

In [1]:
%matplotlib inline
import os
import glob
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal
import traceback
import warnings

from matplotlib import pyplot as plt
from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)


In [2]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util import sglxutil as sglu
from ceciestunepipe.util import rigutil as ru
from ceciestunepipe.util import wavutil as wu
from ceciestunepipe.util import syncutil as su

from ceciestunepipe.util.sound import boutsearch as bs

from ceciestunepipe.util.spikeextractors import preprocess as pre
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import readSGLX as rsgl
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import spikeglxrecordingextractor as sglex

In [3]:
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
logger.info('all modules loaded')

2022-02-15 09:10:34,840 root         INFO     all modules loaded


## Session parameters and raw files

#### list all the sessions for this bird

In [4]:
bird = 's_b1238_22'
all_bird_sess = et.list_sessions(bird)
logger.info('all sessions for bird are {}'.format(all_bird_sess))

2022-02-15 09:10:36,034 root         INFO     all sessions for bird are ['2022-01-07', '2022-01-08', '2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15', '2022-01-16', '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28', '2022-01-29', '2022-01-30', '2022-01-31', '2022-02-01', '2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08', '2022-02-09', '2022-02-10', '2022-02-11', '2022-02-12', '2022-02-13', '2022-02-14', '2022-02-15']


### set up bird and sessions parameters
this will define:
- locations of files (for the bird)
- signals and channels to look for in the metadata of the files and in the rig.json parameter file: Note that this have to exist in all of the sessions that will be processed
- 'sess' is unimportant here, but it comes handy if there is need to debug usin a single session

In [5]:
reload(et)
# for one example session

sess_par = {'bird': 's_b1238_22',
           'sess': '2022-01-27',
           'probes': ['probe_0'], #probes of interest
           'mic_list': ['microphone_0', 'microphone_1'], #list of mics of interest, by signal name in rig.json
           'stim_list': ['wav_stim', 'wav_syn'], # list of adc chans with the stimulus
           'nidq_ttl_list': ['wav_ttl'], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
           'sort': 2, #label for this sort instance
           }

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])

ksort_folder = exp_struct['folders']['ksort']
raw_folder = exp_struct['folders']['sglx']

list all the epochs in a session, to check that it is finding what it has to find

In [6]:
sess_epochs = et.list_sgl_epochs(sess_par)
sess_epochs

2022-02-15 09:10:41,363 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1238_22', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-01-27', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-01-27/sglx', 'kwik': '/scratch/earneodo/s_b1238_22/sglx/kwik/2022-01-27', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1238_22/2022-01-27/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1238_22/sglx/msort/2022-01-27', 'ksort': '/scratch/earneodo/s_b1238_22/sglx/ksort/2022-01-27/2', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/2'}, 'files': {'par': '/scratch/earneodo/s_b1238_22/sglx/ksort/2022-01-27/2/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-01-27/sglx/settings.isf', 'rig': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-01-27/sglx/rig.json', 'kwd': '/scratch/earneodo/s_b123

['0605_g0', '1154_g0', '1722_g0', '2005_g0', '2135_nightstim_g0']

In [7]:
[x for x in sess_epochs if any(y in x for y in ['2', 'g0'])]

['0605_g0', '1154_g0', '1722_g0', '2005_g0', '2135_nightstim_g0']

#### define pre-processing steps for each epoch and for the session

In [8]:
reload(pre)
one_epoch_dict = pre.preprocess_run(sess_par, exp_struct, sess_epochs[0])

2022-02-15 09:10:47,345 ceciestunepipe.util.spikeextractors.preprocess INFO     PREPROCESSING sess 2022-01-27 | epoch 0605_g0
2022-02-15 09:10:47,346 ceciestunepipe.util.spikeextractors.preprocess INFO     getting extractors
2022-02-15 09:10:47,388 ceciestunepipe.util.spikeextractors.preprocess INFO     Got sglx recordings for keys ['nidq', 'lf_0', 'ap_0']
2022-02-15 09:10:47,389 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting microphone channel(s) ['microphone_0', 'microphone_1']
2022-02-15 09:10:50,680 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-02-15 09:10:50,681 ceciestunepipe.util.wavutil INFO     saving (2, 834494107)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/0605_g0/wav_mic.wav
2022-02-15 09:11:01,791 ceciestunepipe.util.wavutil INFO     saving (2, 834494107)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/0605_g0/wav_mic.npy
2022-02-15 09:11:07,359 ceciestunepipe.

In [11]:
### sequentially process all runs of the sessions
def preprocess_session(sess_par: dict):
    logger.info('pre-process all runs of sess ' + sess_par['sess'])
    # get exp struct
    sess_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])
    # list the epochs
    sess_epochs = et.list_sgl_epochs(sess_par)
    logger.info('found epochs: {}'.format(sess_epochs))
    # preprocess all epochs
    epoch_dict_list = []
    for i_ep, epoch in enumerate(sess_epochs):
        try:
            exp_struct = et.sgl_struct(sess_par, epoch)
            one_epoch_dict = pre.preprocess_run(sess_par, exp_struct, epoch)
            epoch_dict_list.append(one_epoch_dict)
        except Exception as exc:
            warnings.warn('Error in epoch {}'.format(epoch), UserWarning)
            logger.info(traceback.format_exc)
            logger.info(exc)
            logger.info('Session {} epoch {} could not be preprocessed'.format(sess_par['sess'], epoch))
        
    return epoch_dict_list

#all_epoch_list = preprocess_session(sess_par)

## Process multiple sessions

In [12]:
sess_list = all_bird_sess
# fist implant, right hemisphere
#sess_list = ['2021-06-24', '2021-06-25', '2021-06-26', '2021-06-27', '2021-06-28', '2021-06-29', '2021-06-30']
sess_list = ['2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20', 
             '2022-01-26', '2022-01-27','2022-01-28', '2022-01-29', '2022-01-30',
            '2022-02-11', '2022-01-12', '2022-01-13']

In [ ]:
all_sess_dict = {}

for one_sess in sess_list[:]:
    sess_par['sess'] = one_sess
    preprocess_session(sess_par)

2022-02-15 09:55:02,419 root         INFO     pre-process all runs of sess 2022-01-17
2022-02-15 09:55:02,421 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1238_22', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-01-17', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-01-17/sglx', 'kwik': '/scratch/earneodo/s_b1238_22/sglx/kwik/2022-01-17', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1238_22/2022-01-17/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-17/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1238_22/sglx/msort/2022-01-17', 'ksort': '/scratch/earneodo/s_b1238_22/sglx/ksort/2022-01-17/2', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-17/sglx/2'}, 'files': {'par': '/scratch/earneodo/s_b1238_22/sglx/ksort/2022-01-17/2/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-01-17/sglx/settings.isf', 'rig': '/mnt/sphere/spee

In [17]:
sess_par

{'bird': 's_b1238_22',
 'sess': '2022-01-27',
 'probes': ['probe_0'],
 'mic_list': ['microphone_0', 'microphone_1'],
 'stim_list': ['wav_stim', 'wav_syn'],
 'nidq_ttl_list': ['wav_ttl'],
 'sort': 2}

In [18]:
# Search bouts

## search bouts for those sessions

In [19]:
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.util import wavutil as wu

from joblib import Parallel, delayed
import pickle
import sys

In [20]:
def sess_file_id(f_path):
    n = int(os.path.split(f_path)[1].split('-')[-1].split('.wav')[0])
    return n


def get_all_day_bouts(sess_par: dict, hparams:dict, n_jobs: int=12, ephys_software='sglx', 
                     parallel=True) -> pd.DataFrame:
    
    logger.info('Will search for bouts through all session {}, {}'.format(sess_par['bird'], sess_par['sess']))
    exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software=ephys_software)

    # get all the paths to the wav files of the epochs of the day   
    source_folder = exp_struct['folders']['derived']
    wav_path_list = et.get_sgl_files_epochs(source_folder, file_filter='*wav_mic.wav')
    wav_path_list.sort()
    logger.info('Found {} files'.format(len(wav_path_list)))
    print(wav_path_list)
    
    get_file_bouts = lambda path: bs.get_epoch_bouts(path, hparams)
    # Go parallel through all the paths in the day, get a list of all the pandas dataframes for each file
    if parallel:
        sess_pd_list = Parallel(n_jobs=n_jobs, verbose=100, prefer='threads')(delayed(get_file_bouts)(i) for i in wav_path_list)
    else:
        sess_pd_list = [get_file_bouts(i) for i in wav_path_list]
    
    #concatenate the file and return it, eventually write to a pickle
    sess_bout_pd = pd.concat(sess_pd_list)
    return sess_bout_pd

def save_auto_bouts(sess_bout_pd, sess_par, hparams):
    exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='bouts_sglx')
    #sess_bouts_dir = os.path.join(exp_struct['folders']['derived'], 'bouts_ceciestunepipe')
    sess_bouts_dir = exp_struct['folders']['derived']

    sess_bouts_path = os.path.join(sess_bouts_dir, hparams['bout_auto_file'])
    hparams_pickle_path = os.path.join(sess_bouts_dir, 'bout_search_params.pickle')

    os.makedirs(sess_bouts_dir, exist_ok=True)
    logger.info('saving bouts pandas to ' + sess_bouts_path)
    sess_bout_pd.to_pickle(sess_bouts_path)

    logger.info('saving bout detect parameters dict to ' + hparams_pickle_path)
    with open(hparams_pickle_path, 'wb') as fh:
        pickle.dump(hparams, fh)

In [21]:
hparams = {
    # spectrogram
    'num_freq':1024, #1024# how many channels to use in a spectrogram #
    'preemphasis':0.97, 
    'frame_shift_ms':5, # step size for fft
    'frame_length_ms':10, #128 # frame length for fft FRAME SAMPLES < NUM_FREQ!!!
    'min_level_db':-55, # minimum threshold db for computing spe 
    'ref_level_db':110, # reference db for computing spec
    #'sample_rate':None, # sample rate of your data
    
    # spectrograms
    'mel_filter': False, # should a mel filter be used?
    'num_mels':1024, # how many channels to use in the mel-spectrogram
    'fmin': 500, # low frequency cutoff for mel filter
    'fmax': 12000, # high frequency cutoff for mel filter
    
    # spectrogram inversion
    'max_iters':200,
    'griffin_lim_iters':20,
    'power':1.5,

    # Added for the searching
    'read_wav_fun': wu.read_wav_chan, # function for loading the wav_like_stream (has to returns fs, ndarray)
    'file_order_fun': sess_file_id, # function for extracting the file id within the session
    'min_segment': 30, # Minimum length of supra_threshold to consider a 'syllable' (ms)
    'min_silence': 2000, # Minmum distance between groups of syllables to consider separate bouts (ms)
    'min_bout': 5000, # min bout duration (ms)
    'peak_thresh_rms': 0.55, # threshold (rms) for peak acceptance,
    'thresh_rms': 0.25, # threshold for detection of syllables
    'mean_syl_rms_thresh': 0.3, #threshold for acceptance of mean rms across the syllable (relative to rms of the file)
    'max_bout': 120000, #exclude bouts too long
    'l_p_r_thresh': 100, # threshold for n of len_ms/peaks (typycally about 2-3 syllable spans
    
    'waveform_edges': 1000, #get number of ms before and after the edges of the bout for the waveform sample
    
    'bout_auto_file': 'bout_auto.pickle', # extension for saving the auto found files
    'bout_curated_file': 'bout_checked.pickle', #extension for manually curated files (coming soon)
    }

In [22]:
sess_list

['2022-01-27']

In [23]:
all_sessions = sess_list[:]
#all_sessions = ['2021-06-15']

for sess in all_sessions:
    sess_par['sess'] = sess
    sess_bout_pd = get_all_day_bouts(sess_par, hparams, parallel=False)
    save_auto_bouts(sess_bout_pd, sess_par, hparams)
    sess_bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts')
    #bouts_to_wavs(sess_bout_pd, sess_par, hparams, sess_bouts_folder)

2022-01-28 15:52:12,817 root         INFO     Will search for bouts through all session s_b1238_22, 2022-01-27
2022-01-28 15:52:12,907 root         INFO     Found 1 files
2022-01-28 15:52:12,908 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/0605_g0/wav_mic.wav


['/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/0605_g0/wav_mic.wav']
tu vieja file /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/0605_g0/wav_mic.wav


2022-01-28 15:52:12,912 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 10 chunks


  0%|          | 0/10 [00:00<?, ?it/s]

2022-01-28 15:58:12,195 ceciestunepipe.util.sound.boutsearch INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/0605_g0/bout_search_params.pickle
2022-01-28 15:58:12,204 ceciestunepipe.util.sound.boutsearch INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/sglx/0605_g0/bout_auto.pickle
2022-01-28 15:58:14,996 root         INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/bouts_sglx/bout_auto.pickle
2022-01-28 15:58:17,800 root         INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-01-27/bouts_sglx/bout_search_params.pickle


In [25]:
sess_bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349 entries, 0 to 75
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start_ms      349 non-null    int64  
 1   end_ms        349 non-null    int64  
 2   start_sample  349 non-null    int64  
 3   end_sample    349 non-null    int64  
 4   p_step        349 non-null    object 
 5   rms_p         349 non-null    float64
 6   peak_p        349 non-null    float64
 7   bout_check    349 non-null    bool   
 8   file          349 non-null    object 
 9   len_ms        349 non-null    int64  
 10  syl_in        349 non-null    object 
 11  n_syl         349 non-null    int64  
 12  peaks_p       349 non-null    object 
 13  n_peaks       349 non-null    int64  
 14  l_p_ratio     349 non-null    float64
 15  waveform      349 non-null    object 
 16  confusing     349 non-null    bool   
dtypes: bool(2), float64(3), int64(7), object(5)
memory usage: 44.3+ KB


In [24]:
np.unique(sess_bout_pd['start_ms']).size

103

# debug

## debug search_bout

In [21]:
## look for a single file
sess = sess_list[0]

exp_struct = et.get_exp_struct(sess_par['bird'], sess, ephys_software='sglx')
source_folder = exp_struct['folders']['derived']
wav_path_list = et.get_sgl_files_epochs(source_folder, file_filter='*wav_mic.wav')
wav_path_list.sort()
logger.info('Found {} files'.format(len(wav_path_list)))
print(wav_path_list)

2021-09-22 15:13:39,371 root         INFO     Found 4 files


['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/1255_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/1740_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/2118_g0/wav_mic.wav']


In [22]:
one_file = wav_path_list[0]

In [ ]:
reload(bs)
epoch_bout_pd, epoch_wav = bs.get_bouts_in_long_file(wav_path_list[0], hparams)

2021-09-22 15:13:45,924 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav


tu vieja file /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav


2021-09-22 15:13:45,962 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 5 chunks


  0%|          | 0/5 [00:00<?, ?it/s]